In [ ]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import LSTM, Dense, Dropout,LeakyReLU
from tensorflow.keras.models import Sequential
from keras.preprocessing.sequence import TimeseriesGenerator
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.pipeline import Pipeline
import joblib

mpl.rcParams['figure.figsize'] = (10, 8)
mpl.rcParams['axes.grid'] = False

In [ ]:
from google.colab import drive
#drive.mount("/content/drive", force_remount=True)
drive.mount("/content/drive")


Mounted at /content/drive


In [ ]:
df=pd.read_csv("/content/drive/MyDrive/nevada_weather.csv",header=0)
df

<ipython-input-64-97f8dd411e42>:1: DtypeWarning: Columns (23) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv("/content/drive/MyDrive/nevada_weather.csv",header=0)


,Year,Month,Day,Hour,Minute,DHI,Temperature,Clearsky DHI,Clearsky DNI,Clearsky GHI,...,Ozone,Relative Humidity,Solar Zenith Angle,Surface Albedo,Pressure,Precipitable Water,Wind Direction,Wind Speed,is_day,Datetime
0,2019.0,1.0,1.0,0.0,0.0,0.0,0.6,0.0,0.0,0.0,...,0.342,72.88,167.61,0.14,954.0,0.4,47.0,0.5,False,1/1/19 0:00
1,2019.0,1.0,1.0,0.0,10.0,0.0,0.5,0.0,0.0,0.0,...,0.342,73.41,167.56,0.14,954.0,0.4,53.0,0.5,False,1/1/19 0:10
2,2019.0,1.0,1.0,0.0,20.0,0.0,0.4,0.0,0.0,0.0,...,0.343,73.94,167.14,0.14,954.0,0.4,60.0,0.5,False,1/1/19 0:20
3,2019.0,1.0,1.0,0.0,30.0,0.0,0.4,0.0,0.0,0.0,...,0.343,73.94,166.38,0.14,954.0,0.4,67.0,0.5,False,1/1/19 0:30
4,2019.0,1.0,1.0,0.0,40.0,0.0,0.3,0.0,0.0,0.0,...,0.344,73.73,165.33,0.14,954.0,0.4,74.0,0.5,False,1/1/19 0:40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52565,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52566,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
52568,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df = df[df['is_day'] == True]

In [ ]:
df_input=df.iloc[:,[14,5,12,6,16,17]]
print (df_input)
df_input.describe()

         GHI   DHI    DNI  Temperature  Relative Humidity  Solar Zenith Angle
42       0.0   0.0    0.0         -1.0              70.25               92.53
43       0.0   0.0    0.0         -0.3              66.76               90.13
44      19.0  12.0  281.0          0.3              63.92               88.58
45      40.0  18.0  403.0          0.9              61.21               86.94
46      65.0  24.0  499.0          1.5              66.56               85.27
...      ...   ...    ...          ...                ...                 ...
52511  155.0  50.0  562.0         14.0              59.76               79.22
52512  124.0  56.0  422.0          6.3              79.29               80.78
52513   96.0  49.0  362.0          5.9              81.51               82.41
52514   70.0  40.0  291.0          5.5              83.88               84.07
52515   47.0  31.0  212.0          5.0              86.84               85.73

[26792 rows x 6 columns]


,GHI,DHI,DNI,Temperature,Relative Humidity,Solar Zenith Angle
count,26792.000000,26792.000000,26792.000000,26792.000000,26792.000000,26792.000000
mean,458.437108,101.895603,586.651090,20.351030,45.274218,57.757347
std,316.965486,75.703278,349.834949,8.536349,25.102348,20.734482
min,0.000000,0.000000,0.000000,-2.000000,3.400000,11.950000
25%,171.000000,64.000000,264.000000,13.400000,22.767500,42.857500
50%,441.000000,93.000000,738.000000,19.600000,41.830000,59.005000
75%,725.000000,113.000000,893.000000,27.000000,64.662500,73.812500
max,1063.000000,498.000000,1025.000000,39.700000,100.000000,99.330000


In [ ]:

import numpy as np
data_array = np.array(df_input)
print(data_array)
features=data_array
target=data_array[:,0]

[[ 0.000e+00  0.000e+00  0.000e+00 -1.000e+00  7.025e+01  9.253e+01]
 [ 0.000e+00  0.000e+00  0.000e+00 -3.000e-01  6.676e+01  9.013e+01]
 [ 1.900e+01  1.200e+01  2.810e+02  3.000e-01  6.392e+01  8.858e+01]
 ...
 [ 9.600e+01  4.900e+01  3.620e+02  5.900e+00  8.151e+01  8.241e+01]
 [ 7.000e+01  4.000e+01  2.910e+02  5.500e+00  8.388e+01  8.407e+01]
 [ 4.700e+01  3.100e+01  2.120e+02  5.000e+00  8.684e+01  8.573e+01]]


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(features, target, test_size=0.40, random_state=None, shuffle = False)
print (x_train.shape)
print (x_test.shape)
print (y_train.shape)
print (y_test.shape)

(16075, 6)
(10717, 6)
(16075,)
(10717,)


In [ ]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
y_train=x_train[:,0]
x_test = scaler.transform(x_test)
y_test=x_test[:,0]
print (x_train)
print (x_test)

[[-1.43448723 -1.28635834 -1.57074586 -2.41483206  0.78654067  1.68605801]
 [-1.43448723 -1.28635834 -1.57074586 -2.33013211  0.63847997  1.5765363 ]
 [-1.37661989 -1.14842828 -0.78643998 -2.25753215  0.51799505  1.50580353]
 ...
 [-0.61825319 -0.53923718  0.26023156  1.55396587 -1.59685476  0.7487347 ]
 [-0.72789656 -0.57371969  0.15695997  1.46926592 -1.5306729   0.84000279]
 [-0.83144864 -0.63119055  0.04810613  1.38456596 -1.50352137  0.93035821]]
[[-0.93500072 -0.68866141 -0.08028558  1.311966   -1.47933954  1.02025728]
 [-1.03550715 -0.74613227 -0.22821516  1.22726604 -1.45006679  1.10970001]
 [-1.12992229 -0.82659147 -0.40126485  1.15466608 -1.42418799  1.19823006]
 ...
 [-1.14210488 -0.72314392 -0.56035893 -1.5799325   1.26423794  1.22424146]
 [-1.22129177 -0.82659147 -0.75852874 -1.62833247  1.36478345  1.29999398]
 [-1.2913417  -0.93003901 -0.97902754 -1.68883244  1.49035929  1.3757465 ]]


In [ ]:
win_length=50
batch_size=432
num_features=6
train_generator = TimeseriesGenerator(x_train, y_train, length=win_length, sampling_rate=1, batch_size=batch_size)
test_generator = TimeseriesGenerator(x_test, y_test, length=win_length, sampling_rate=1, batch_size=batch_size)
#train_generator[0]
print (test_generator[0])

(array([[[-0.93500072, -0.68866141, -0.08028558,  1.311966  ,
         -1.47933954,  1.02025728],
        [-1.03550715, -0.74613227, -0.22821516,  1.22726604,
         -1.45006679,  1.10970001],
        [-1.12992229, -0.82659147, -0.40126485,  1.15466608,
         -1.42418799,  1.19823006],
        ...,
        [ 1.68425776, -0.11395282,  1.13385341,  1.8806657 ,
         -1.73430939, -1.71687282],
        [ 1.69644036, -0.11395282,  1.13664454,  1.90486569,
         -1.73940031, -1.74470958],
        [ 1.70253166, -0.11395282,  1.13664454,  1.92906568,
         -1.74406698, -1.75976882]],

       [[-1.03550715, -0.74613227, -0.22821516,  1.22726604,
         -1.45006679,  1.10970001],
        [-1.12992229, -0.82659147, -0.40126485,  1.15466608,
         -1.42418799,  1.19823006],
        [-1.21824612, -0.8955565 , -0.60222579,  1.08206612,
         -1.39703646,  1.28539109],
        ...,
        [ 1.69644036, -0.11395282,  1.13664454,  1.90486569,
         -1.73940031, -1.74470958],
 

In [ ]:
pipeline= Sequential()
pipeline.add(LSTM(64, input_shape= (win_length, num_features), return_sequences=True))
pipeline.add(LeakyReLU(alpha=0.5))
pipeline.add(LSTM(32, return_sequences=False))
pipeline.add(LeakyReLU(alpha=0.5))
pipeline.add(Dropout(0.02))
pipeline.add(Dense(1))
pipeline.summary()

Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_12 (LSTM)              (None, 50, 64)            18176     
                                                                 
 leaky_re_lu_12 (LeakyReLU)  (None, 50, 64)            0         
                                                                 
 lstm_13 (LSTM)              (None, 32)                12416     
                                                                 
 leaky_re_lu_13 (LeakyReLU)  (None, 32)                0         
                                                                 
 dropout_6 (Dropout)         (None, 32)                0         
                                                                 
 dense_6 (Dense)             (None, 1)                 33        
                                                                 
Total params: 30625 (119.63 KB)
Trainable params: 3062

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
pipeline.compile(loss=MeanSquaredError(), optimizer=Adam(), metrics=[RootMeanSquaredError()])
pipeline.fit(train_generator, validation_data=test_generator, epochs=50, callbacks=[early_stopping])

joblib.dump(scaler,"scaler.joblib")
pipeline.save("model.h5")

Epoch 1/50
38/38 [==============================] - 8s 57ms/step - loss: 0.2937 - root_mean_squared_error: 0.5419 - val_loss: 0.0594 - val_root_mean_squared_error: 0.2438
Epoch 2/50
38/38 [==============================] - 1s 38ms/step - loss: 0.0701 - root_mean_squared_error: 0.2648 - val_loss: 0.0327 - val_root_mean_squared_error: 0.1809
Epoch 3/50
38/38 [==============================] - 1s 33ms/step - loss: 0.0565 - root_mean_squared_error: 0.2377 - val_loss: 0.0260 - val_root_mean_squared_error: 0.1614
Epoch 4/50
38/38 [==============================] - 1s 34ms/step - loss: 0.0533 - root_mean_squared_error: 0.2310 - val_loss: 0.0214 - val_root_mean_squared_error: 0.1463
Epoch 5/50
38/38 [==============================] - 1s 23ms/step - loss: 0.0519 - root_mean_squared_error: 0.2278 - val_loss: 0.0219 - val_root_mean_squared_error: 0.1481
Epoch 6/50
38/38 [==============================] - 1s 23ms/step - loss: 0.0500 - root_mean_squared_error: 0.2236 - val_loss: 0.0209 - val_root_m

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
pipeline.evaluate_generator(test_generator, verbose=0)
predictions=pipeline.predict_generator(test_generator)
predictions.shape[0]

<ipython-input-73-58f2686263ec>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  pipeline.evaluate_generator(test_generator, verbose=0)
<ipython-input-73-58f2686263ec>:2: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  predictions=pipeline.predict_generator(test_generator)


10667

In [ ]:
print (y_test)
print (x_test)
print (x_test[:,1:][win_length:])
df_pred=pd.concat([pd.DataFrame(predictions), pd.DataFrame(x_test[:,1:][win_length:])],axis=1)
print (df_pred)

[-0.93500072 -1.03550715 -1.12992229 ... -1.14210488 -1.22129177
 -1.2913417 ]
[[-0.93500072 -0.68866141 -0.08028558  1.311966   -1.47933954  1.02025728]
 [-1.03550715 -0.74613227 -0.22821516  1.22726604 -1.45006679  1.10970001]
 [-1.12992229 -0.82659147 -0.40126485  1.15466608 -1.42418799  1.19823006]
 ...
 [-1.14210488 -0.72314392 -0.56035893 -1.5799325   1.26423794  1.22424146]
 [-1.22129177 -0.82659147 -0.75852874 -1.62833247  1.36478345  1.29999398]
 [-1.2913417  -0.93003901 -0.97902754 -1.68883244  1.49035929  1.3757465 ]]
[[-0.11395282  1.13943566  1.94116567 -1.74661243 -1.76205052]
 [-0.11395282  1.13664454  1.94116567 -1.74661243 -1.75155469]
 [-0.11395282  1.13664454  1.94116567 -1.74661243 -1.72782499]
 ...
 [-0.72314392 -0.56035893 -1.5799325   1.26423794  1.22424146]
 [-0.82659147 -0.75852874 -1.62833247  1.36478345  1.29999398]
 [-0.93003901 -0.97902754 -1.68883244  1.49035929  1.3757465 ]]
              0         0         1         2         3         4
0      1.648652

In [ ]:
rev_trans=scaler.inverse_transform(df_pred)
print (rev_trans)
df_final=df_input[predictions.shape[0]*-1:]
print(df_final.count())
df_final['GHI_Pred']=rev_trans[:,0]
print (df_final)

[[1012.30925162  102.          971.           35.           10.54
    16.97      ]
 [1010.83986517  102.          970.           35.           10.54
    17.2       ]
 [1007.70613174  102.          970.           35.           10.54
    17.72      ]
 ...
 [  97.00807869   49.          362.            5.9          81.51
    82.41      ]
 [  70.37985764   40.          291.            5.5          83.88
    84.07      ]
 [  48.83669768   31.          212.            5.           86.84
    85.73      ]]
GHI                   10667
DHI                   10667
DNI                   10667
Temperature           10667
Relative Humidity     10667
Solar Zenith Angle    10667
dtype: int64
          GHI    DHI    DNI  Temperature  Relative Humidity  \
30313  1030.0  102.0  971.0         35.0              10.54   
30314  1029.0  102.0  970.0         35.0              10.54   
30315  1026.0  102.0  970.0         35.0              10.54   
30316  1019.0  105.0  964.0         35.1              10.13   


<ipython-input-75-c55eb0babe22>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['GHI_Pred']=rev_trans[:,0]


In [ ]:
actual_values = df_final['GHI']
predicted_values = df_final['GHI_Pred']
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from math import sqrt
rmse = sqrt(mean_squared_error(actual_values, predicted_values))
r2 = r2_score(actual_values, predicted_values)
print("Root Mean Squared Error (RMSE):", rmse)
print("R2 Score:",r2)

Root Mean Squared Error (RMSE): 39.37339143118214
R2 Score: 0.9825016216882552
